Scripts for regression experiments on mouse

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
from functools import cmp_to_key, partial
import re

import mp_run
import conf_interval

from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error, explained_variance_score

import os

from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from scipy import stats

from multiprocessing import Pool, cpu_count


# regex for number extraction from string
number_pattern =  r'(-?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+-]?\d+)?)'


In [2]:
train_source_1 = pd.read_csv('../data/mouse/GSE115553/normalized/train_source.csv.gz', index_col=0, compression='gzip')
train_target_1 = pd.read_csv('../data/mouse/GSE115553/normalized/train_target.csv.gz', index_col=0, compression='gzip')
test_source_1 = pd.read_csv('../data/mouse/GSE115553/normalized/test_source.csv.gz', index_col=0, compression='gzip')
test_target_1 = pd.read_csv('../data/mouse/GSE115553/normalized/test_target.csv.gz', index_col=0, compression='gzip')

train_source_2 = pd.read_csv('../data/mouse/GSE151173/normalized/train_source.csv.gz', index_col=0, compression='gzip')
train_target_2 = pd.read_csv('../data/mouse/GSE151173/normalized/train_target.csv.gz', index_col=0, compression='gzip')
test_source_2 = pd.read_csv('../data/mouse/GSE151173/normalized/test_source.csv.gz', index_col=0, compression='gzip')
test_target_2 = pd.read_csv('../data/mouse/GSE151173/normalized/test_target.csv.gz', index_col=0, compression='gzip')

train_source_3 = pd.read_csv('../data/mouse/GSE171975/normalized/train_source.csv.gz', index_col=0, compression='gzip')
train_target_3 = pd.read_csv('../data/mouse/GSE171975/normalized/train_target.csv.gz', index_col=0, compression='gzip')
test_source_3 = pd.read_csv('../data/mouse/GSE171975/normalized/test_source.csv.gz', index_col=0, compression='gzip')
test_target_3 = pd.read_csv('../data/mouse/GSE171975/normalized/test_target.csv.gz', index_col=0, compression='gzip')

In [4]:
common_exp_genes = list(set(train_source_1.index).intersection(set(train_source_2.index), set(train_source_3.index)))

train_source_1 = train_source_1.loc[common_exp_genes]
train_target_1 = train_target_1.loc[common_exp_genes]
test_source_1 = test_source_1.loc[common_exp_genes]
test_target_1 = test_target_1.loc[common_exp_genes]

train_source_2 = train_source_2.loc[common_exp_genes]
train_target_2 = train_target_2.loc[common_exp_genes]
test_source_2 = test_source_2.loc[common_exp_genes]
test_target_2 = test_target_2.loc[common_exp_genes]

train_source_3 = train_source_3.loc[common_exp_genes]
train_target_3 = train_target_3.loc[common_exp_genes]
test_source_3 = test_source_3.loc[common_exp_genes]
test_target_3 = test_target_3.loc[common_exp_genes]

In [5]:
train_source = pd.concat([train_source_1, train_source_2, train_source_3], axis=1)
train_target = pd.concat([train_target_1, train_target_2, train_target_3], axis=1)
test_source = pd.concat([test_source_1, test_source_2, test_source_3], axis=1)
test_target = pd.concat([test_target_1, test_target_2, test_target_3], axis=1)

In [7]:
network_df = pd.read_csv('../data/mouse/regnetworkweb.org.network', sep='\t')
regulator_set = set(network_df['regulator'])
target_set = set(network_df['target'])

regulator_set = regulator_set.intersection(set(train_source.index))
target_set = target_set.intersection(set(train_source.index))
all_gene_set = regulator_set.union(target_set)
network_dict = {target: [] for target in target_set}
for ind, row in network_df.iterrows():
    if (row['regulator'] in regulator_set) and (row['target'] in target_set):
        network_dict[row['target']].append(row['regulator'])

key_list = []
value_list = []
regulator_set = set()
tf_list_df = pd.read_csv('../data/mouse/mouse_tf_list.tsv.gz', sep='\t', compression='gzip', index_col=0)
for name in tf_list_df['Gene Names']:
    name_splits = str(name).split(' ')
    for i in name_splits:
        if i in train_source.index:
            regulator_set.add(i)
target_set = set()
for key in network_dict.keys():
    if (len(network_dict[key]) > 0) and network_dict[key][0] != key:
        key_list.append(key)
        target_set.add(key)
        value_list.append("; ".join(network_dict[key]))
        for regulator in network_dict[key]:
            regulator_set.add(regulator)
all_gene_set = regulator_set.union(target_set)

print('Number of TFs used:')
print(len(regulator_set))

Number of TFs used:
1385


In [6]:
# network_df = pd.read_csv('../data/mouse/regnetworkweb.org.network', sep='\t')
# regulator_set = set(network_df['regulator'])
# target_set = set(network_df['target'])

# regulator_set = regulator_set.intersection(set(train_source.index))
# target_set = target_set.intersection(set(train_source.index))
# all_gene_set = regulator_set.union(target_set)
# network_dict = {target: [] for target in target_set}
# for ind, row in network_df.iterrows():
#     if (row['regulator'] in regulator_set) and (row['target'] in target_set):
#         network_dict[row['target']].append(row['regulator'])

# key_list = []
# value_list = []
# regulator_set = set()
# target_set = set()
# for key in network_dict.keys():
#     if (len(network_dict[key]) > 0) and network_dict[key][0] != key:
#         key_list.append(key)
#         target_set.add(key)
#         value_list.append("; ".join(network_dict[key]))
#         for regulator in network_dict[key]:
#             regulator_set.add(regulator)
# all_gene_set = regulator_set.union(target_set)


# print(len(regulator_set))

In [8]:
network_df = pd.DataFrame(index=key_list)
network_df['tf_list'] = value_list
target_df = pd.concat([train_target, test_target], axis=1)
source_df = pd.concat([train_source, test_source], axis=1)


target_gene_list = list(target_set)
target_exp = target_df.loc[target_gene_list]
X = source_df.loc[list(regulator_set)]
tf_list = list(regulator_set)


tf_list_df = pd.read_csv('../output/network_model/mouse_tf.csv', names=['tf'], index_col=0)
tf_list = list(tf_list_df.index)

In [9]:
# GS stats
edge_count = 0
gs_tf_set = set()
gs_target_set = set()
for i, row in network_df.iterrows():
    if i in target_gene_list:
        cur_tf_list = row.tf_list
        gs_target_set.add(i)
        if pd.isnull(cur_tf_list): 
            continue
        cur_tf_list = cur_tf_list.split('; ')
        for cur_tf in cur_tf_list:
            if cur_tf in tf_list:
                gs_tf_set.add(cur_tf)
                edge_count += 1
print('GS edge count:')
print(edge_count)
print('Number of TFs in GS:')
print(len(gs_tf_set))
print('Number of target genes in GS:')
print(len(gs_target_set))



GS edge count:
40331
Number of TFs in GS:
780
Number of target genes in GS:
8211


In [10]:
# filter for high variance targets
new_test_target = test_target.loc[target_gene_list]
new_test_target = new_test_target.loc[new_test_target.std(axis=1) > 0.5]
target_gene_list = new_test_target.index

In [11]:
print('training set size:')
print(train_source.shape)
print('testing set size:')
print(test_source.shape)

training set size:
(9931, 208)
testing set size:
(9931, 121)


In [13]:
mp_calc = mp_run.MpCalc(target_gene_list, X, network_df, train_source.loc[tf_list], train_target, test_source.loc[tf_list], test_target)

In [14]:
iter_length = len(target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))

  0%|          | 0/310 [00:00<?, ?it/s]100%|██████████| 310/310 [01:13<00:00,  4.24it/s]


In [15]:
r = np.array(r)
out_df = pd.DataFrame(index=target_gene_list)
out_df['rf_score'] = r[:, 0]
out_df['linear_score'] = r[:, 1]
out_df['gs_rf_score'] = r[:, 2]
out_df['gs_linear_score'] = r[:, 3]
out_df['rf_with_linear_top_features_score'] = r[:, 4]
out_df['linear_with_rf_top_features_score'] = r[:, 5]
out_df['rf_rmse'] = r[:, 6]
out_df['linear_rmse'] = r[:, 7]
out_df['gs_rf_rmse'] = r[:, 8]
out_df['gs_linear_rmse'] = r[:, 9]
out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
out_df['linear_with_rf_top_features_rmse'] = r[:, 11]
out_df['rf_with_top_features_score'] = r[:, 12]
out_df['linear_with_top_features_score'] = r[:, 13]
out_df['rf_with_top_features_rmse'] = r[:, 14]
out_df['linear_with_top_features_rmse'] = r[:, 15]
out_df['rf_top_feature_num'] = r[:, 16]
out_df['linear_top_feature_num'] = r[:, 17]
out_df['rf_top_features_gs_overlap'] = r[:, 18]
out_df['linear_top_features_gs_overlap'] = r[:, 19]
out_df['rf_linear_top_features_overlap'] = r[:, 20]
out_df['gs_edge_num'] = r[:, 21]
out_df['test_var'] = r[:, 22]
out_df['test_std'] = r[:, 23]
out_df['pca_rf_score'] = r[:, 24]
out_df['pca_rf_rmse'] = r[:, 25]

In [17]:
tf_list_df = pd.DataFrame(index=tf_list)
tf_list_df.to_csv('../output/network_model/mouse_tf.csv', header=False)
out_df.to_csv('../output/network_model/mouse_all_tf_high_var_target_new.csv.gz', compression='gzip')

In [14]:
out_df = pd.read_csv('../output/network_model/mouse_all_tf_high_var_target_new.csv.gz', index_col=0, compression='gzip')
iter_length = len(target_gene_list)
new_out_df = pd.DataFrame(index=target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.rf_top_tf_same_count_as_gs, range(iter_length)), total=iter_length))
efron_r = np.array(r)
new_out_df['rf_top_tf_same_count_as_gs_score'] = efron_r[:, 0]
new_out_df['rf_top_tf_same_count_as_gs_rmse'] = efron_r[:, 1]
new_out_df = new_out_df.loc[out_df.index]
out_df['rf_top_tf_same_count_as_gs_score'] = new_out_df['rf_top_tf_same_count_as_gs_score']
out_df['rf_top_tf_same_count_as_gs_rmse'] = new_out_df['rf_top_tf_same_count_as_gs_rmse']

100%|██████████| 310/310 [00:22<00:00, 13.93it/s]


In [15]:
out_df.to_csv('../output/network_model/mouse_all_tf_high_var_target_new.csv.gz', compression='gzip')

In [14]:
out_df = pd.read_csv('../output/network_model/mouse_all_tf_high_var_target_new.csv.gz', index_col=0, compression='gzip')
iter_length = len(target_gene_list)
new_out_df = pd.DataFrame(index=target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.efron_process_rf_training, range(iter_length)), total=iter_length))
efron_r = np.array(r)
new_out_df['rf_efron_feature_num'] = efron_r[:, 0].astype('float64')
new_out_df['rf_efron_complementary_feature_num_list'] = efron_r[:, 1]
new_out_df['rf_efron_rmse'] = efron_r[:, 2].astype('float64')
new_out_df['rf_efron_complementary_rmse_list'] = efron_r[:, 3]
new_out_df['rf_efron_features'] = efron_r[:, 4]
new_out_df['rf_efron_complementary_features_list'] = efron_r[:, 5]
new_out_df['rf_efron_ensemble_rmse'] = efron_r[:, 6]
new_out_df = new_out_df.loc[out_df.index]
out_df['rf_efron_feature_num'] = new_out_df['rf_efron_feature_num']
out_df['rf_efron_complementary_feature_num_list'] = new_out_df['rf_efron_complementary_feature_num_list']
out_df['rf_efron_rmse'] = new_out_df['rf_efron_rmse']
out_df['rf_efron_complementary_rmse_list'] = new_out_df['rf_efron_complementary_rmse_list']
out_df['rf_efron_features'] = new_out_df['rf_efron_features']
out_df['rf_efron_complementary_features_list'] = new_out_df['rf_efron_complementary_features_list']
out_df['rf_efron_ensemble_rmse'] = new_out_df['rf_efron_ensemble_rmse']

out_df.to_csv('../output/network_model/mouse_all_tf_high_var_target_efron_train.csv.gz', compression='gzip')

100%|██████████████████████████████████████████████████████████████████████████████| 310/310 [11:24<00:00,  2.21s/it]


In [ ]:
tf_list_df = pd.read_csv('../output/network_model/mouse_tf.csv', names=['tf'], index_col=0)
out_df = pd.read_csv('../output/network_model/mouse_all_tf_high_var_target_new.csv.gz', index_col=0, compression='gzip')
available_tfs = set(X.index)
rf_efron_overlap_count = []
for target_gene in out_df.index:
    gs_tf_list = network_df.loc[target_gene].tf_list
    gs_tf_set = set(gs_tf_list.split('; '))
    gs_tf_set = available_tfs.intersection(gs_tf_set)
    if target_gene in gs_tf_set: gs_tf_set.remove(target_gene)
    efron_tf_list = out_df.loc[target_gene]['rf_efron_features']
    efron_tf_list = efron_tf_list.split('; ')
    efron_tf_list = [int(i) for i in efron_tf_list]
    efron_tf_list = tf_list_df.iloc[efron_tf_list].index
    efron_tf_set = set(efron_tf_list)
    rf_efron_overlap_count.append(len(efron_tf_set.intersection(gs_tf_set)))
out_df['rf_efron_overlap_count'] = rf_efron_overlap_count
out_df.to_csv('../output/network_model/mouse_all_tf_high_var_target_new.csv.gz', compression='gzip')


In [14]:
out_df = pd.read_csv('../output/network_model/mouse_all_tf_high_var_target_efron_train.csv.gz', index_col=0, compression='gzip')
iter_length = len(target_gene_list)
new_out_df = pd.DataFrame(index=target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.rf_top_10, range(iter_length)), total=iter_length))
top10_r = np.array(r)
new_out_df['rf_top10_score'] = top10_r[:, 0].astype('float64')
new_out_df['rf_top10_rmse'] = top10_r[:, 1].astype('float64')
new_out_df = new_out_df.loc[out_df.index]
out_df['rf_top10_score'] = new_out_df['rf_top10_score']
out_df['rf_top10_rmse'] = new_out_df['rf_top10_rmse']
out_df.to_csv('../output/network_model/mouse_all_tf_high_var_target_efron_train.csv.gz', compression='gzip')

100%|██████████| 310/310 [00:25<00:00, 12.28it/s]
